In [1]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 69.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 57.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 76.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 94.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.5 MB/s eta 0:00:0000:0

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import gc

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

In [2]:
!nvidia-smi

Thu Oct 26 18:08:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   49C    P0    73W / 300W |  11755MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
df = pd.read_csv('./vseros-rank-otb/train_df.tsv',sep='\t')
df['pair'] = df['community_id'] + '[SEP]' + df['customer_id']
counts = df.groupby('pair').transform('count')
df['recs'] = counts['Unnamed: 0']

In [3]:
df_rank = df[['community_id','customer_id','recs']]
df_rank = df_rank.drop_duplicates()

In [4]:
users_inv_mapping = dict(enumerate(df['customer_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

items_inv_mapping = dict(enumerate(df['community_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(users_mapping),len(items_mapping)

(1568446, 149114)

In [5]:
all_set = set(df_rank.index)
test_idx = list( set(np.load('./test_index.npy')) & all_set)
train_gbm_idx = list( all_set & set(np.load('./train_gbm_index.npy')) )
train_idx = list(all_set - (set(test_idx) | set(train_gbm_idx)) )

In [6]:
test_df = df_rank.loc[test_idx]
train_df_gbm = df_rank.loc[train_gbm_idx]
train_df = df_rank.loc[train_idx]

In [8]:
test_df

,community_id,customer_id,recs
12058625,1af19bf7fb2783b065781a6348dca98bd496706f8d634e...,fe9116d83575ac37c543e49241ad6a894c8ea3dc626833...,1
6291458,ee1f320c066d6614c79585d6c17a5c4e43471404bdb635...,6dfe1246304aee642c5054404321b69865f338df2b9298...,2
9437190,fbb19e0798df299ef6958b161d3e2ab9d0f79579b1cf1e...,7b617381354a0445e1e0022aac3f9d81c5fec45152101b...,1
9961478,4a2ff719779fc792d29f2272a6c25ef51c50260955e318...,f1d24161f7a0f56efbd85b40eb15e661e97721cf1313c2...,1
1572872,d858329a98955cf0c9940c1d51f7a09eeada28a5374f05...,4b25a2bf93ee6d395465911003627c51a92d5aebee20bc...,1
...,...,...,...
10485746,69f7ce8667c7cb7eb985a518c2977bb72686bab103d391...,4046bc278abb0773fb2bada3bfd83c7b99765c61de93a7...,1
7864307,6be63805549fc080e95fad89c70dab7a6c3a153b083bfa...,544bad357bc6668002db6dd42c277312b2ef700493ea49...,1
11010039,ef33891f96e39d5a4ac98c529fde5a7f69baad9fcf92fb...,91709254ce9fde6096b9b6312d3e27adaa50758f9afab4...,1
9437176,fbb19e0798df299ef6958b161d3e2ab9d0f79579b1cf1e...,8fa31cc9428b3209707d44ae53f49336fc19c3800132c6...,1


In [9]:
del df
gc.collect()

0

In [41]:
model = SAR(
    col_user="customer_id",
    col_item="community_id",
    col_rating="recs",
    #col_timestamp="timestamp",
    similarity_type="jaccard", 
    #time_decay_coefficient=30, 
    #timedecay_formula=True,
    normalize=True
)

In [ ]:
model.fit(train_df)

In [10]:
dataset = Dataset()

In [14]:
dataset.fit(users=train_df['customer_id'], 
            items=train_df['community_id'])

In [26]:
np.stack([train_df['customer_id'].values,
                                                               train_df['community_id'].values,
                                                               train_df['recs'].values]).shape

(3, 10496708)

In [24]:
(interactions, weights) = dataset.build_interactions(np.stack([train_df['customer_id'].values,
                                                               train_df['community_id'].values,
                                                               train_df['recs'].values]))

ValueError: Expecting tuples of (user_id, item_id, weight) or (user_id, item_id). Got ['947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1b7ce5a43f58d9dba320c133d58b3ca781fac348fa4beaec4e39b9717168b3df67a1d187a706523cbf2'
 'b935c3390d82df612de19591d2dd16e1eff8e660746bb3a9e954b173e9d7e9421b86466c1e5995a97e6fc289d0b8c8d0b17ba9ba04913b1703e590a090761cc5'
 'ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3f290e52639266b1510c07013df80b3379df53e74c9c41b81e966294c9dba52c5398e3802610b9c9ff0'
 ...
 'a8b43d4a5e9583edac068333ce010c559e03e6a33dbbfbd856ded696d3f7a3d34d35a3f8cc9b0c60eb8bbf554ee70b320fe30a6ab7a1ceaef10105fe70fa3445'
 'e70cced119a8f54a9f1bdcc4979735ebff7d817797c236991402a9befb5ff78f70998ad261c4ef33598afcd33118be5b52b00959229c301bfd12dfd6609ad596'
 '83caecb16267482f439fedcf9106706bccd15095fe564ace299b3eb0186edee6cea830a94e6a454d5e660a4fff04ec53535277c68824c4202f6cf590071a180e']

In [16]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [27]:
train_group = train_df[['community_id','customer_id']].groupby('customer_id').agg(lambda x:x.tolist())

KeyboardInterrupt: 

In [ ]:
sorted_tr_data = train_df.sort_values(by='customers_count')[::-1]
best_subs = sorted_tr_data.community_id.unique()[:200]
simple_preds = best_subs[:7].tolist()
mapk(act,[simple_preds] * len(act))

In [74]:
act = test_group['community_id'].tolist()
mapk(act,rmk_preds.tolist(),k=7)

0.00011528911564625848

In [77]:
rmk_preds[1]

array(['8d7d59076ae7149f7887be1700f5cdbcb8b41d8395a22ab107007047a5463b51fe2463440f11e8f69cbdd4acdff97a78968cabcfc044745b25d6131551824bdb',
       'acde8a186a309ea3b457ed9a0c98cacb71b62fbad47bfd8fa41aea535a82bc2cb7889ebdc4a0d33cf27e85fcdcf18c1284ceab54386a78f20b62b4ab8c02ca4a',
       'c17e17f6eca305de49ced3c6b5472924ab71c94a0d7519ca9981005eb0651cc8927023cd5098b203d2fd6e9e8087b83201697b50b43598816a5bf76e624dd181',
       'ec9d0673bb132410c8441176d4a4ea18d5024a7f02a38de14758ccf8d36882995f8d1fc814f398d3e2238eb00c72fb7fb1d6085747243935a4beabb7dfab462f',
       '0a546c7b54fb9a0ed2a1a60d3e02e9417ee80984e1c6d38293aeab5671dcf2b1ac2c6006264d87acc654eadfe174e75140b3b93b129b98a139e0ced890add8a1',
       '11c447c7cbf20b664224f9db0b7661c88efc255f9e52112424a0e5021d23081095148159644d792e2bbdbeccfd16cb65df608239e5078a7a9b1255f8262eebac',
       'db149b8bcf227aa8b587185c74abb70894a2380a6e8143987c197b9d55f272f5e47b6f582ba1b8de07b5cd6312c8a0a7ecdf02b0f01552edba818c9aa0c74421'],
      dtype='<U128')

In [79]:
test_preds[1]

['1193c191061a52e851adbaa550cd1de8e40a5d11ee0a714b3b316331d86572262bddf6ab2c31ec79554ed745c482cf0cd52ee1fc53426ee6d45423396104a585',
 'f873b83cd2d9eaf3e5e1d181e41bcbe2108df665f644f103443948896ec458ff03a61e22a5f90ec840c5006fe1f200d076f51c155b983257bc0a6513a87a6b0a',
 '3a495d488a80c25dfe789992a0f51b257bcd935694ad19ef8e0d50066c1883942d6dd43bb0fc037ace9527b1f7402bd15cfd218e5c5a523b3d3dc78a7de23d70',
 '55d99312f32b4817ecdcb160db1cd19b090e0acf28deb4ed368ba0a4a8469d4d7280e0642d67f755c25a5eadc7c4508eb83a37082ea125365822484f1cc8a78b',
 'e707ddeb5c57a9789a727f15236355440a17aa28a47b9b0fc0f067bda2f69ee543e14329a7975e6b42aeb7eb77f61b2d7fd773df72f1c19c0e8cea1b92aa32f7',
 '03cb8f26c3eb07bb6eb174673f353940403b39ffa4ccb6f7fe0d6d1a2d2019406682f97cebefd44dd9070341252b698025a7b58225a4c7f6e90c4f37af2137fa',
 'fca646973562ee10a7a9a947e059504994d24d5f9ea6a279f4be27d9778a5d4fdbd3163efbebdd3c7cda43d06b06ad8b788b3282f87e6d53af5ec4d3fb7d8a5d',
 '1d93267e648bf5d7eb87bc93da4463aaff7f45451a6edd601b3a81ca40f4610c860

In [49]:
test_preds[0]

['fa0b3fca5d767521d3aceae5f71c601467cd1874f2f213153ea35f6e8bc98acfbb936a8b618a8506ee0abb121f46d58eb3415d5b8f9c87e9769d94a4f0708422',
 '788d6f1380f41cf88eef1ccfdb73902c6a4618007797f75422e7c7273cc236a94ce92d731e62190f894c0af684bbc5fdbe4346dead4624b61045f9455a5d7c00',
 '6bfd7f576af5b5dd14ef85d43ae1839908002bbc4ad5c775b2798806e361599087660bec82d4238f26685c9fbb2d7f4508843774c257f0cd536606f6e82ab5fd',
 'b0273c85ae7731959b30db3cf59828855b63af0d6da46e1aed2004eb0e2f2d07d971ed951f73842083dd8542062c99b26f3649549f1d09fb3ff0228e3b02dcf6',
 '961bf396baa08be47f080972bab6b5ac0c2b36706152e41fdf5053c9851fef7260bb5945c0401dec2aa02ca565fd93a2e92fe964adcd28567ee3945da68075aa',
 '9d0361a322e3964c6a23f533c52f6cfb74c665d7019cc161672bd11331f6218d4e208729ba8ee58f6a871f35475ba1f1bbc6ce993c9042293c61a275cd931969',
 'dd873be43c0e3315eec04e9cb32eef6c5eb15b6d52811d044e20b0e4f89e26188486485f296a0b24f5e185bc9d8fdb06bd106dd754f0dbe15959599f11dd0c07',
 '036d5c4792bfa5405f0c81e522630a333419087ed8238fc76b1ff834ae005266104

In [32]:
train_gbm_group = train_df_gbm[['community_id','customer_id']].groupby('customer_id').agg(lambda x:x.tolist())

In [33]:
gbm_preds,gbm_scores = predict_impl(model,
                                    train_gbm_group.index,
                                    train_mat,
                                    users_mapping,
                                    items_inv_mapping,
                                    N=50,
                                    falh=True)

  0%|          | 0/160000 [00:00<?, ?it/s]

In [35]:
act = train_gbm_group['community_id'].tolist()
mapk(gbm_preds,act,k=7)

0.035478129251700685

In [34]:
rank_df = get_df(train_gbm_group.index,gbm_preds,gbm_scores)

In [36]:
rank_df = set_label(rank_df,train_gbm_group)

  0%|          | 0/8000000 [00:00<?, ?it/s]

In [37]:
rank_df

,query,user_id,scor,comunity,label
0,0,00002894054dc36c3aea6f27fe41c9b0e458fd270abe0b...,2.172693,6d8c10905790df73c0c19a42a42e001e2755d6534c4822...,0
1,0,00002894054dc36c3aea6f27fe41c9b0e458fd270abe0b...,2.016075,fa0b3fca5d767521d3aceae5f71c601467cd1874f2f213...,0
2,0,00002894054dc36c3aea6f27fe41c9b0e458fd270abe0b...,1.995933,7b487366ee754c82461c55763c5e89973dd2f3c72a007e...,0
3,0,00002894054dc36c3aea6f27fe41c9b0e458fd270abe0b...,1.815209,fa6bf8e9f0367db7159d7224490944b8f7bbda45eb4b37...,0
4,0,00002894054dc36c3aea6f27fe41c9b0e458fd270abe0b...,1.759100,c8830341f1341984d1be04aacb86e36244ddf962f40a98...,0
...,...,...,...,...,...
7999995,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.392810,d8af7321a0f9d04a60a69b6e755d52f64499155da78b74...,0
7999996,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.385779,67c9e87c4837c795d86e78de4d7d486867c3ab2848263b...,0
7999997,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.385735,3638604316576f10bb866b0c1e2831f18006b8faabd640...,0
7999998,159999,ffffea67894044381d713871f6cb6d482a45c031b1272f...,0.383474,f1d44c0b58ac76fba036ae50117cadeabf0e19eaf38992...,0


In [38]:
rank_df.to_parquet('train_gbm.parquet')

In [39]:
test_pr_df = get_df(test_group.index,test_preds,test_scores)
test_pr_df.to_parquet('test_gbm.parquet')